# Content-Based Movie Recommendation

Dataset Source:

https://www.kaggle.com/datasets/jrobischon/wikipedia-movie-plots/

In [178]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer

In [179]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [180]:
df = pd.read_csv("../data/wiki_movie_plots_deduped.csv")
df.shape

(34886, 8)

In [181]:
df.sample(5)

Release Year                          Title Origin/Ethnicity  \
23374          1994              A Chinese Odyssey        Hong Kong   
4143           1945     Thunderhead, Son of Flicka         American   
9010           1974               Black Belt Jones         American   
597            1926  The Great K & A Train Robbery         American   
22377          2015             Driving with Selvi         Canadian   

             Director                                          Cast  \
23374     Jeffrey Lau          Stephen Chow, Athena Chu, Ng Man-tat   
4143       Louis King  Roddy McDowall, Preston Foster, Rita Johnson   
9010    Robert Clouse                      Jim Kelly, Gloria Hendry   
597      Lewis Seiler                         Tom Mix, Dorothy Dwan   
22377  Elisa Paloschi                                           NaN   

               Genre  \
23374        unknown   
4143   family, drama   
9010         unknown   
597          western   
22377    documentary   

                                                             Wiki Page  \
23374                  https://en.wikipedia.org/wiki/A_Chinese_Odyssey   
4143   https://en.wikipedia.org/wiki/Thunderhead,_Son_of_Flicka_(film)   
9010                    https://en.wikipedia.org/wiki/Black_Belt_Jones   
597      https://en.wikipedia.org/wiki/The_Great_K_%26_A_Train_Robbery   
22377                 https://en.wikipedia.org/wiki/Driving_with_Selvi   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

## Converting to String

In [182]:
df.columns

Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')

In [183]:
df = df.astype(str)
df.dtypes

Release Year        object
Title               object
Origin/Ethnicity    object
Director            object
Cast                object
Genre               object
Wiki Page           object
Plot                object
dtype: object

## Data Cleaning

Action Items:

* Lower-Case the whole data frame
* Director: Removing 'Director:' and 'Cast:'
* Director, Cast: Removing '\r\n', '\n' and '\r'

* Genre: Replacing '/' with Space
* Director, Cast, Genre: Removing 'Uknonwn' and 'Nan'

* Director: Separating Directors and Actors names
* Director, Cast: Checking if the names are separated with ' and ', ' & '

* Director, Cast: Merging the first names and last names together
* Director, Cast: Adding the words of 'Director' and 'Actor' as prefix

* Plot: Removing English Stopwords
* Doc: Removing special characters

In [184]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-184-614eaff662ae>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [185]:
df["Director"] = df["Director"].str.replace("director:", "", regex=False)
df["Director"] = df["Director"].str.replace("cast:", "", regex=False)

df["Director"] = df["Director"].str.replace("\r\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\r\n", " ", regex=False)

df["Director"] = df["Director"].str.replace("\n", " ", regex=False)
df["Cast"] = df["Cast"].str.replace("\n", " ", regex=False)

df["Genre"] = df["Genre"].str.replace("/", " ", regex=False)

df["Director"] = df["Director"].str.replace("unknown", "", regex=False)
df["Cast"] = df["Cast"].str.replace("unknown", "", regex=False)
df["Genre"] = df["Genre"].str.replace("unknown", "", regex=False)

df["Director"] = df["Director"].str.replace("nan", "", regex=False)
df["Cast"] = df["Cast"].str.replace("nan", "", regex=False)
df["Genre"] = df["Genre"].str.replace("nan", "", regex=False)


df["Director"] = df["Director"].str.replace(" and ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" and ", ",", regex=False)
df["Director"] = df["Director"].str.replace(" & ", ",", regex=False)
df["Cast"] = df["Cast"].str.replace(" & ", ",", regex=False)


df["Director"] = df["Director"].str.replace(" ", "", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", "", regex=False)

df["Director"] = df["Director"].str.replace(",", " ", regex=False)
df["Cast"] = df["Cast"].str.replace(",", " ", regex=False)


In [186]:
df["Director"] = np.where(df["Director"].str.len() > 0,
                          'director' + df["Director"],
                          df["Director"])

df["Cast"] = np.where(df["Cast"].str.len() > 0,
                      'actor' + df["Cast"],
                      df["Cast"])

df["Director"] = df["Director"].str.replace(" ", " director", regex=False)
df["Cast"] = df["Cast"].str.replace(" ", " actor", regex=False)


In [187]:
df.sample(5)

Release Year                 Title Origin/Ethnicity  \
3984          1945             china sky         american   
34481         2010  the man from nowhere     south_korean   
8791          1972                 frogs         american   
23181         1985        my lucky stars        hong kong   
20784         2001                 hotel          british   

                    Director  \
3984      directorrayenright   
34481  directorleejeong-beom   
8791   directorgeorgemccowan   
23181      directorsammohung   
20784     directormikefiggis   

                                                                     Cast  \
3984                   actorrandolphscott actorruthwarrick actorellendrew   
34481                                         actorwonbin actorkimsae-ron   
8791                                      actorraymilland actorsamelliott   
23181                        actorsammohung actorjackiechan actoryuenbiao   
20784  actorsaffronburrows actorsalmahayek actorlucyliu actorburtreynolds   

           Genre                                                  Wiki Page  \
3984   war drama             https://en.wikipedia.org/wiki/china_sky_(film)   
34481             https://en.wikipedia.org/wiki/the_man_from_nowhere_(film)   
8791      horror                 https://en.wikipedia.org/wiki/frogs_(film)   
23181     action               https://en.wikipedia.org/wiki/my_lucky_stars   
20784      drama            https://en.wikipedia.org/wiki/hotel_(2001_film)   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Merging the document

In [188]:
df["doc"] = df[['Release Year', 'Title', 'Origin/Ethnicity',
                'Director', 'Cast', 'Genre', 'Plot']].agg(' '.join, axis=1)
df.sample(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc
20713,1999,faeries,british,directorgaryhurst,actorkatewinslet actorjeremyirons,animated fantasy,https://en.wikipedia.org/wiki/faeries_(1999_film),"while waiting for their new home to be renovated, nellie and her younger brother george are sent to a farm in the countryside, much to george's delight and nellie's disgust. however, the farmhouse and the surrounding area are teeming with fey creatures. the first the two children encounter is a somewhat unfriendly hobgoblin named broom who is (more or less) secretly looking after the farm.\r\nwhile playing outside, george inadvertently stumbles into a fairy ring surrounding a tree and ends up in the realm of faeries. nellie, alerted by broom, goes after him but arrives just in time to find george eating from a faery cake. the law of the realm states that a mortal who consumes faery food must remain in the realm forever. nellie and george strongly — and vociferously — protest. so the faery prince, the ruler of the realm, offers them a chance to escape by setting three tasks for them.\r\nwhat they do not know is that the evil brother of the prince, the shapeshifter, tries to manipulate the children and the fairy citizens to usurp the rulership of the faery realm. in addition, the prince falls in love with the human farmhand brigid, which proves pivotal in an old prophecy that foretells the future for the faery realm.","1999 faeries british directorgaryhurst actorkatewinslet actorjeremyirons animated fantasy while waiting for their new home to be renovated, nellie and her younger brother george are sent to a farm in the countryside, much to george's delight and nellie's disgust. however, the farmhouse and the surrounding area are teeming with fey creatures. the first the two children encounter is a somewhat unfriendly hobgoblin named broom who is (more or less) secretly looking after the farm.\r\nwhile playing outside, george inadvertently stumbles into a fairy ring surrounding a tree and ends up in the realm of faeries. nellie, alerted by broom, goes after him but arrives just in time to find george eating from a faery cake. the law of the realm states that a mortal who consumes faery food must remain in the realm forever. nellie and george strongly — and vociferously — protest. so the faery prince, the ruler of the realm, offers them a chance to escape by setting three tasks for them.\r\nwhat they do not know is that the evil brother of the prince, the shapeshifter, tries to manipulate the children and the fairy citizens to usurp the rulership of the faery realm. in addition, the prince falls in love with the human farmhand brigid, which proves pivotal in an old prophecy that foretells the future for the faery realm."
23333,1992,the shootout,hong kong,,,,https://en.wikipedia.org/wiki/the_shootout_(film),"hon (elvis tsui), hei, koon and bonnie (kingdom yuen) are a group of bank robbers who hijacked an armoured truck. hon and hei managed to escape from the cops while koon and bonnie encounter rookie cop wong ka-fai (aaron kwok), whom arrests koon when he fell off his motorcycle. knowing that one of his accomplices were arrested, hon bursts into the police station and murders koon to prevent divulgence of his crimes.\r\nsince this case is serious, ka-fai and his colleague mei-san (ngai suet) were transferred to the regional crime unit to assist in investigation, where they collaborate with inpsector lau (sean lau) and ma (bryan leung), whom are frenemies. first, they investigate the armoured truck guards before finding clues that lead them to a karaoke bar, where ka-fai falls in love with an innocent singer named man (fennie yuen). soon later, ka-fai found clues about the hijack case from man, who turns out to be hon's girlfriend.","1992 the shootout hong kong hon (elvis tsui), hei, koon and bonnie (kingdom yuen) are a group of bank robbers who hijacked an armoured truck. hon and hei managed to escape from the c

## Removing Special Characters

In [189]:
df["doc"] = df["doc"].str.replace("[^a-z 0-9]+", "", regex=True)

In [190]:
pd.set_option('display.max_colwidth', None)
df[["doc"]].sample(10)

doc
14382                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  2003 willard american directorglenmorgan actorcrispinglover horror willard stiles crispin glover is a social misfit taking care of his ill and fragile but verbally abusive mother henrietta jackie burroughs in a musty old mansion that is also home to a colony of rats willard finds himself constantly humiliated in front of his coworkers by his cruel boss frank martin r lee ermey a vicious man who assumed control of the company from willards father upon his death by suicide and whose professional interest in willard extends to a personal financial one a coworker cathryn laura elena harring has sympathy for the quirky willard2 cathryn becomes willards friend and love interestwillard quickly becomes obsessed with his friendship with a white rat he names socrates willard then begins to train and befriend the other rats including an extra large one he names ben ben begins to assume a position of leadership among the other rats while socrates remains willards favorite after he trains the rats sufficiently willard takes them to martins home where he orders them to chew up the tires on martins mercedesbenz willards mother panics when she overhears the rats and later dies by falling down the stairs of the basement now socrates is all he has left willard learns soon afterward that the payments on the house have fallen far behind and that the bank will likely foreclose upon the propertydesperately lonely willard begins to bring socrates to work with him he then finds a note at his desk declaring that he is being fired by frank martin from the company his father founded while he is arguing with martin begging not to be fired socrates is discovered by ms leach in the supply room her screams alert martin who bludgeons socrates to death willard his mental state already precarious is devastated especially since he knows that he could have done nothing to prevent socrates death from happening he turns to ben who is more than willing to guide the colony of rats to help willard avenge himself upon his boss willard and his rats confront martin and upon willards command t

## Removing Stopwords

In [191]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords


In [192]:
stops = stopwords.words('english')
print(stops)

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [193]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

df['doc_clean'] = df['doc'].map(lambda s:preprocess(s)) 

In [195]:
df[["doc", 'doc_clean']].sample(10)

doc  \
1666                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 1934 the count of monte cristo american directorrowlandvlee actorrobertdonat actorelissalandi actorlouiscalhern drama adventure in 1815 a french merchant ship stops at the island of elba a letter from the exiled napoleon is given to the ships captain to deliver to a man in marseille before he dies of a sickness the captain entrusts the task to his first officer edmond dants donat however the city magistrate raymond de villefort jr calhern is tipped off by an informer the second officer danglars raymond walburn and has both men arrested after the exchangedants friend fernand mondego sidney blackmer accompanies him to the jail however he danglars and de villefort all stand to gain from keeping dants imprisoned mondego is in love with dants fiance mercedes landi danglars wants to be promoted captain in dants place and the man who accepted the letter turns out to be de villeforts father lawrence grant de villefort consigns dants without trial to a notorious prison the chteau dif on the false testimony of danglarswhen napoleon returns to france giving dants friends hope for his release de villefort signs a false statement that he was killed trying to escape which mondego shows to mercedes deceived she gives in to her mothers deathbed wish and marries mondegoeight years of solitary confinement follow for dants then one day the aged abb faria o p heggie a fellow prisoner breaks into his cell through a tunnel he has been digging the two join forces faria calculates it will take five more years to finish in the meantime he starts educating dantshowever as they near their goal a cavein fatally injures the old man before he dies he bequeaths a vast hidden treasure to his protg farias enemies had tortured and imprisoned him in an unsuccessful attempt to extract its location the body is sewn into a shroud but while the undertaker is away dants substitutes himself for the corpse undetected he is cast into the sea he frees himself and is picked up by a smuggling shipdants later follows the abbs directions and finds the treasure on the uninhabited island of monte cristo with a fortune at his command he sets in motion his plans for revenge to begin he arranges to have albert mercedes and mondegos son kidnapped and held for ransom dants rescues the younger man in order to gain entry into paris society using his purchased title of count of monte cristofirst to be brought to justice is mondego while the french ambassador to albania mondego gained renown for his bravery in an unsuccessful defense of ali pasha dants arranges a ball to honor his enemy then arranges to have him exposed publicly as the one who betrayed ali pasha to his death at the hands of the turks unaware of the counts role in his disgrace mondego goes to him for advice dants reveals his identity and they engage in a duel dants wins but spares mondego who 

## TF-IDF

In [196]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["doc_clean"])
column_names = vectorizer.get_feature_names_out()

df_tf_idf = pd.DataFrame(X.toarray(), columns=column_names)
df_tf_idf.shape

(34886, 251188)

## Cosine Similarity

In [197]:
from sklearn.metrics.pairwise import cosine_similarity

df_cos_sim = pd.DataFrame(cosine_similarity(df_tf_idf, dense_output=True))
df_cos_sim.shape

/opt/miniconda3/envs/ai/lib/python3.9/site-packages/sklearn/utils/extmath.py:193: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


(34886, 34886)

In [198]:
df_cos_sim.to_parquet("../data/movie_cos_sim.parquet")

## Testing

In [203]:
pd.set_option('display.max_colwidth', 50)


In [204]:
query = 'titanic'

df[df["Title"].str.contains("titanic")]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc,doc_clean
6008,1953,titanic,american,directorjeannegulesco,actorbarbarastanwyck actorcliftonwebb actorrob...,biography,https://en.wikipedia.org/wiki/titanic_(1953_film),"at the last minute, a wealthy american expatri...",1953 titanic american directorjeannegulesco ac...,1953 titan american directorjeannegulesco acto...
9758,1980,raise the titanic,american,directorjerryjameson,actorjasonrobards actoralecguinness actorannea...,action,https://en.wikipedia.org/wiki/raise_the_titani...,the film opens on the fictional island of svar...,1980 raise the titanic american directorjerryj...,1980 rais titan american directorjerryjameson ...
12857,1996,titanic,american,directorrobertlieberman,actorgeorgec.scott actorevamariesaint actorpet...,biography,https://en.wikipedia.org/wiki/titanic_(1996_tv...,titanic follows three main story threads.\r\ni...,1996 titanic american directorrobertlieberman ...,1996 titan american directorrobertlieberman ac...
13153,1997,titanic,american,directorjamescameron,actorleonardodicaprio actorkatewinslet actorbi...,"historical epic, disaster",https://en.wikipedia.org/wiki/titanic_(1997_film),"in 1996, treasure hunter brock lovett and his ...",1997 titanic american directorjamescameron act...,1997 titan american directorjamescameron actor...
16392,2012,titanic 3d,american,directorjamescameron,actorleonardodicaprio actorkatewinslet actorbi...,drama,https://en.wikipedia.org/wiki/titanic_(1997_film),"in 1996, treasure hunter brock lovett and his ...",2012 titanic 3d american directorjamescameron ...,2012 titan american directorjamescameron actor...


In [207]:
df_titanic = df_cos_sim[[13153]] # titanic movie
df_titanic.shape

(34886, 1)

In [215]:
df_titanic.head()

,13153
0,0.016928
1,0.003581
2,0.000865
3,0.008599
4,0.250213


In [209]:
df_titanic.max()

13153    1.0
dtype: float64

In [217]:
df_titanic.sort_values(by=[13153]).tail(5)


,13153
6275,0.398865
16004,0.421042
14599,0.472291
16392,0.992128
13153,1.000000


In [223]:
df[df.index == 6275]

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot,doc,doc_clean
6275,1955,east of eden,american,directoreliakazan,actorjulieharris actorjamesdean actorraymondma...,drama,https://en.wikipedia.org/wiki/east_of_eden_(film),"the story is set during 1917 and 1918, leading...",1955 east of eden american directoreliakazan a...,1955 east eden american directoreliakazan acto...
